In [11]:
# cnn : 이미지 한장한장을 학습하는데 유리
# rnn : 시계열 데이터를 학습하는데 유리
# 언어나 영상같은 이미지 등에도 적용하기 좋음


import tensorflow as tf
import numpy as np

tf.compat.v1.reset_default_graph()
# RNN keep weights in the memory
# 값을 초기화 시켜줘서 

idx2char = ['h','i','e','l','o']
x_data = [[0,1,0,2,3,3]]
# 입력할 단어 -> h i h e l l
y_data = [[1,0,2,3,3,4]]
# 예측할 단어 -> i h e l l o

x_one_hot = [[[1,0,0,0,0], # h 0
             [0,1,0,0,0],  # i 1
             [1,0,0,0,0],  # h 0
             [0,0,1,0,0],  # e 2
             [0,0,0,1,0],  # l 3
             [0,0,0,1,0]]] # l 3
# 문자는 수치값이 없어서 예측하기 매우 어렵다
# 따라서 벡터라이즈 해준다



num_class = 5

input_dim = 5

batch_size = 1

sequence_length = 6
# cell의 갯수

learning_rate = 0.1

X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])
Y = tf.placeholder(tf.int32, [None, sequence_length])
# y는 인덱스값을 예측하려고 해서 int로 넣음

cell = tf.contrib.rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
# LSTM에 가중치에 관한 모든게 들어가있음
# 노드갯수 5 다음스테이트로 넘어가는거 true
# x원핫 (1,5) * 가중치 (5, x ) = (1,x) 가 되는데
# 이때 x에 해당하는 값이 num_units

init_state = cell.zero_state(batch_size, tf.float32)
#초기 스테이트는 0으로 받겠다
# [0,0,0,0,0] 으로 받겠다



outputs, _state = tf.nn.dynamic_rnn(cell, X, initial_state = init_state, dtype = tf.float32)


X_for_fc = tf.reshape(outputs, [-1, 5])
outputs = tf.contrib.layers.fully_connected(inputs=X_for_fc,
                                           num_outputs=num_class,
                                           activation_fn=None)

# activation_fn이 sigmoid나 relu가 아닌 none이면 linear regression 하겠다는 뜻
# 여기서 activation을 안넣은 이유는 0,1,2,3,4의 인덱스값을 리턴해야하기때문에



outputs = tf.reshape(outputs, [batch_size, sequence_length, num_class])

weight = tf.ones([batch_size, sequence_length])
# ones : 1로 채워라
# 이 가중치는 실제 데이터와 예측 데이터간에 차이를 구할때 한 시퀀스 안에 각각의 값에 대해 얼마나 가중치를 둘것인가

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs,
                                                targets = Y,
                                                weights = weight)
# seq2seq : [0.8, 0.2, 0.7, -1, 3, 3] 과 [1,0,2,3,3,4] 의 시퀀스간 거리


loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis = 2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X:x_one_hot, Y:y_data})
        result = sess.run(prediction, feed_dict={X:x_one_hot})
        print(i,' loss ',l, ' prediction ', result, ' Y:  ',y_data )
        
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print(' \tPrediction string : ',''.join(result_str))

0  loss  1.6038203  prediction  [[3 3 3 3 3 3]]  Y:   [[1, 0, 2, 3, 3, 4]]
 	Prediction string :  llllll
1  loss  1.5147628  prediction  [[3 3 3 3 3 3]]  Y:   [[1, 0, 2, 3, 3, 4]]
 	Prediction string :  llllll
2  loss  1.423818  prediction  [[3 3 3 3 3 3]]  Y:   [[1, 0, 2, 3, 3, 4]]
 	Prediction string :  llllll
3  loss  1.337347  prediction  [[3 3 3 3 3 4]]  Y:   [[1, 0, 2, 3, 3, 4]]
 	Prediction string :  lllllo
4  loss  1.2325989  prediction  [[3 3 3 3 3 4]]  Y:   [[1, 0, 2, 3, 3, 4]]
 	Prediction string :  lllllo
5  loss  1.1055201  prediction  [[2 2 3 3 3 4]]  Y:   [[1, 0, 2, 3, 3, 4]]
 	Prediction string :  eelllo
6  loss  0.98688823  prediction  [[2 2 2 3 3 4]]  Y:   [[1, 0, 2, 3, 3, 4]]
 	Prediction string :  eeello
7  loss  0.864604  prediction  [[2 2 2 3 3 4]]  Y:   [[1, 0, 2, 3, 3, 4]]
 	Prediction string :  eeello
8  loss  0.77505237  prediction  [[2 2 2 3 3 4]]  Y:   [[1, 0, 2, 3, 3, 4]]
 	Prediction string :  eeello
9  loss  0.68510276  prediction  [[2 0 2 3 3 4]]  Y:   [

In [49]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
# mnist를 rnn을 통해 딥러닝

In [51]:
tf.compat.v1.reset_default_graph()

input_dim = 28
sequence_length = 28
num_class = 10
batch_size = 128
hidden_layer_size = 128
training_epochs = 10

X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])
Y = tf.placeholder(tf.float32, [None, num_class])


rnn_cell = tf.contrib.rnn.BasicRNNCell(hidden_layer_size)
outputs, _ = tf.nn.dynamic_rnn(rnn_cell, X, dtype=tf.float32)


W = tf.Variable(tf.truncated_normal([hidden_layer_size, num_class],
                                    mean=0, stddev=0.01))
b = tf.Variable(tf.truncated_normal([num_class],
                                    mean=0, stddev=0.01))
# truncated : -1 이하, 1이상을 짤라버림




last_outputs = outputs[:,-1,:]
# (n,784) 로 바꿔줌

hypothesis = tf.matmul(last_outputs, W) + b


softmax = tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y)
cost = tf.reduce_mean(softmax)
train = tf.train.AdamOptimizer(0.001).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape([batch_size, sequence_length, input_dim])
            feed_dict = {X:batch_xs, Y:batch_ys}
            c, _ = sess.run([cost, train], feed_dict=feed_dict)
            avg_cost += c / total_batch
        print("Epoch: {}, Cost: {}".format(epoch+1, avg_cost))
    print("Learning Finished") 
    
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    test_data = mnist.test.images[:batch_size].reshape((-1,sequence_length, input_dim))
    test_label = mnist.test.labels[:batch_size]
   
    print("Accuracy of Test data: {}".format(sess.run(accuracy, feed_dict={X:test_data,
                                                             Y:test_label})))

Epoch: 1, Cost: 0.6756589013240828
Epoch: 2, Cost: 0.2721867805698535
Epoch: 3, Cost: 0.21016050758825885
Epoch: 4, Cost: 0.17866291010226973
Epoch: 5, Cost: 0.15424367975191772
Epoch: 6, Cost: 0.14235541267463783
Epoch: 7, Cost: 0.12906600953468347
Epoch: 8, Cost: 0.12235905398508325
Epoch: 9, Cost: 0.11201202152144941
Epoch: 10, Cost: 0.11104967416913218
Learning Finished
Accuracy of Test data: 1.0


In [52]:
# rnn - 셀이 넘어가면서 이전 데이터에 대한 영향이 점점 줄어듬
# h1이 x1에서 인풋받아서 h2로 전달
# h2에서는 x2값과 h1값을 인풋으로 받아서
#  h2 = tanh(   w1*h1 + w2*x2   +   b )
#  y2 =  w * h2  +  b
# 이런모양이 되면서
# h1의 값의 영향력이 줄어들고, h3에서 h1의 영향력은 더더욱 줄어들기에
# LSTM을 만듬


# LSTM - 이전 데이터에 대해 영향력을 높힘
# forget 함수 - 시그모이드 함수와 h1, x2를 사용하여 이전 데이터를 받아올지, 잊어버릴지 결정
# 이후 x2의 tanh( ) 와 더하여 영향력을 유지해줌

In [ ]:
#                     https://github.com/gilbutITbook/080228
# 에서 예제 다운로드 후
# 아나콘다 프롬프트에서 텐서플로우를 2.0.0으로 업그레이드

# venv\Scripts\activate

# pip install --upgrade Tensorflow==2.0.0